In [1]:
pip install ultralytics


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\abi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
!yolo task=detect mode=train model=yolov8n.pt data=data.yaml epochs=50 imgsz=640


^C


Ultralytics 8.3.119  Python-3.10.0 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width


  0%|          | 0.00/755k [00:00<?, ?B/s]
 68%|██████▊   | 512k/755k [00:00<00:00, 4.73MB/s]
100%|██████████| 755k/755k [00:00<00:00, 6.03MB/s]

train: Scanning F:\proj\fish\data\train\labels...:   0%|          | 0/876 [00:00<?, ?it/s]
train: Scanning F:\proj\fish\data\train\labels... 22 images, 0 backgrounds, 0 corrupt:   3%|▎         | 22/876 [00:00<00:03, 217.37it/s]
train: Scanning F:\proj\fish\data\train\labels... 84 images, 0 backgrounds, 0 corrupt:  10%|▉         | 84/876 [00:00<00:01, 448.71it/s]
train: Scanning F:\proj\fish\data\train\labels... 145 images, 0 backgrounds, 0 corrupt:  17%|█▋        | 145/876 [00:00<00:01, 518.09it/s]
train: Scanning F:\proj\fish\data\train\labels... 201 images, 0 backgrounds, 0 corrupt:  23%|██▎       | 201/876 [00:00<00:01, 531.28it/s]
train: Scanning F:\proj\fish\data\train\labels... 262 images, 0 backgrounds, 0 corrupt:  30%|██▉       | 262/876 [00:00<00:01, 550.86it/s]
train: Scanning F:\proj\fish\data\train\labels... 325 images, 0 backgro

In [10]:
!yolo task=detect mode=predict model=runs\detect\train\weights\best.pt source='13567_2013_Article_234_Fig2_HTML.jpg'

Ultralytics 8.3.119  Python-3.10.0 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
Model summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 f:\proj\fish\13567_2013_Article_234_Fig2_HTML.jpg: 480x640 1 1- BGD, 90.2ms
Speed: 3.5ms preprocess, 90.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict2
 Learn more at https://docs.ultralytics.com/modes/predict


In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO

# Load your model
model = YOLO('runs/detect/train/weights/best.pt')  # <-- put your model path here

# Mapping acronyms to full disease names
full_name_mapping = {
    'BDA': 'Bacterial Disease of Aeromonas',
    'BGD': 'Bacterial Gill Disease',
    'BRD': 'Bacterial Red Disease',
    'FDS': 'Fungal Disease Syndrome',
    'HF': 'Hemorrhagic Fever',
    'PD': 'Parasitic Disease',
    'WTD': 'White Tail Disease'
}

# Create the main window
root = tk.Tk()
root.title("Fish Disease Detection")
root.geometry("1000x650")
root.configure(bg="lightblue")

# Panels for images
panelA = tk.Label(root)
panelA.pack(side="left", padx=10, pady=10)

panelB = tk.Label(root)
panelB.pack(side="right", padx=10, pady=10)

# Frame for result text and scrollbar
text_frame = tk.Frame(root, bg="lightblue")
text_frame.pack(side="bottom", fill="x", padx=20, pady=20)

# Scrollbar
scrollbar = tk.Scrollbar(text_frame)
scrollbar.pack(side="right", fill="y")

# Text widget for displaying result
label_result = tk.Text(text_frame, height=10, width=80, font=("Arial", 12), wrap="word", yscrollcommand=scrollbar.set, bg="white")
label_result.pack(side="left", fill="both", expand=True)
label_result.configure(state='disabled')
scrollbar.config(command=label_result.yview)

# Function to select and predict image
def select_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Load and display original image
        orig_img = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        orig_img = cv2.resize(orig_img, (400, 400))
        orig_img = ImageTk.PhotoImage(Image.fromarray(orig_img))
        panelA.config(image=orig_img)
        panelA.image = orig_img

        # Predict
        results = model.predict(file_path, save=True, save_dir=None)
        result = results[0]

        # Get predicted labels
        labels_text = ""
        if result.boxes is not None:
            for box in result.boxes:
                cls_id = int(box.cls[0])
                conf = box.conf[0]
                short_name = result.names[cls_id]  # Example: '1- BGD'
                acronym = short_name.split('-')[1].strip()  # Extract 'BGD'
                full_name = full_name_mapping.get(acronym, acronym)
                labels_text += f"{full_name} ({conf:.2f})\n"

        # Update text widget
        label_result.configure(state='normal')
        label_result.delete(1.0, tk.END)
        label_result.insert(tk.END, labels_text)
        label_result.configure(state='disabled')

        # Load predicted image and display
        pred_img = result.plot()  # draw bounding boxes
        pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)
        pred_img = cv2.resize(pred_img, (400, 400))
        pred_img = ImageTk.PhotoImage(Image.fromarray(pred_img))
        panelB.config(image=pred_img)
        panelB.image = pred_img

# Button to select image
btn = tk.Button(root, text="Select an Image", command=select_image, font=("Arial", 14))
btn.pack(side="top", pady=20)

# Start GUI
root.mainloop()



image 1/1 F:\proj\fish\13567_2013_Article_234_Fig2_HTML.jpg: 480x640 1 1- BGD, 136.7ms
Speed: 3.9ms preprocess, 136.7ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict15

image 1/1 F:\proj\fish\52AD7DC1-E132-4520-A93B-622AF1495141-1024x1024.jpg: 640x640 1 0- BDA, 2 1- BGDs, 121.6ms
Speed: 4.6ms preprocess, 121.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict15
